<a href="https://colab.research.google.com/github/dcpatton/Structured-Data/blob/main/target_encoding_cms_claims.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective
This notebook is a proof of concept. It shows an example of Target Encoding categorical data.

In [ ]:
import tensorflow as tf
import pandas as pd
pd.set_option('display.max_rows', 999)
pd.set_option('max_info_columns', 200)
import numpy as np
import random

seed=52
tf.random.set_seed(seed)
random.seed(seed)

tf.__version__

'2.3.0'

# Get the Data

The data can be downloaded from https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/DE_Syn_PUF. Here I retrieve it from my GCP storage.

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com >/dev/null

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   443  100   443    0     0  12657      0 --:--:-- --:--:-- --:--:-- 12305


In [ ]:
!gcloud init --skip-diagnostics --account='dcpatton@gmail.com' --project='cms-de1' 

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: dcpatton@gmail.com
  project: cms-de1

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You are logged in as: [dcpatton@gmail.com].

Your current project has been set to: [cms-de1].

Not setting default zone/region (this feature makes it easier to use
[gcloud compute] by setting an appropriate default value for the
--zone and --region flag).
See https://cloud.google.com/compute/docs/gcloud-compute section on how to set
default compute region and zone manually. If you would like [gcloud init] to be
able to do this for you the next time you run it, make sure the
Compute Engine API i

In [ ]:
!gsutil cp gs://de-synpuf/*.zip .

Copying gs://de-synpuf/176537_DE1_0_2010_Beneficiary_Summary_File_Sample_20.zip...
Copying gs://de-synpuf/176541_DE1_0_2008_Beneficiary_Summary_File_Sample_1.zip...
Copying gs://de-synpuf/176549_DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.zip...
Copying gs://de-synpuf/176600_DE1_0_2009_Beneficiary_Summary_File_Sample_1.zip...
- [4 files][ 12.8 MiB/ 12.8 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://de-synpuf/176616_DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.zip...
Copying gs://de-synpuf/DE1_0_2008_to_2010_Carrier_Claims_Sample_1A.zip...
Copying gs://de-synpuf/DE1_0_2008_to_2010_Carrier_Claims_Sample_1B.zip...
Copying gs://de-synpuf/DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.zip...
- [8 files][361.5 MiB/36

# EDA

Examining the inpatient claims and beneficiary summary files.

In [ ]:
!unzip 176549_DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.zip

Archive:  176549_DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.zip
  inflating: DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv  


In [ ]:
claims_df = pd.read_csv('DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv', parse_dates=['CLM_FROM_DT', 'CLM_THRU_DT'])

In [ ]:
claims_df.head()

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,OT_PHYSN_NPI,CLM_ADMSN_DT,ADMTNG_ICD9_DGNS_CD,CLM_PASS_THRU_PER_DIEM_AMT,NCH_BENE_IP_DDCTBL_AMT,NCH_BENE_PTA_COINSRNC_LBLTY_AM,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,CLM_UTLZTN_DAY_CNT,NCH_BENE_DSCHRG_DT,CLM_DRG_CD,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,ICD9_DGNS_CD_3,ICD9_DGNS_CD_4,ICD9_DGNS_CD_5,ICD9_DGNS_CD_6,ICD9_DGNS_CD_7,ICD9_DGNS_CD_8,ICD9_DGNS_CD_9,ICD9_DGNS_CD_10,ICD9_PRCDR_CD_1,ICD9_PRCDR_CD_2,ICD9_PRCDR_CD_3,ICD9_PRCDR_CD_4,ICD9_PRCDR_CD_5,ICD9_PRCDR_CD_6,HCPCS_CD_1,HCPCS_CD_2,HCPCS_CD_3,HCPCS_CD_4,...,HCPCS_CD_6,HCPCS_CD_7,HCPCS_CD_8,HCPCS_CD_9,HCPCS_CD_10,HCPCS_CD_11,HCPCS_CD_12,HCPCS_CD_13,HCPCS_CD_14,HCPCS_CD_15,HCPCS_CD_16,HCPCS_CD_17,HCPCS_CD_18,HCPCS_CD_19,HCPCS_CD_20,HCPCS_CD_21,HCPCS_CD_22,HCPCS_CD_23,HCPCS_CD_24,HCPCS_CD_25,HCPCS_CD_26,HCPCS_CD_27,HCPCS_CD_28,HCPCS_CD_29,HCPCS_CD_30,HCPCS_CD_31,HCPCS_CD_32,HCPCS_CD_33,HCPCS_CD_34,HCPCS_CD_35,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45
0,00013D2EFD8E45D1,196661176988405,1,2010-03-12,2010-03-13,2600GD,4000.0,0.0,3.139084e+09,NaN,NaN,20100312,4580,0.0,1100.0,0.0,0.0,1.0,20100313,217,7802,78820,V4501,4280,2720,4019,V4502,73300,E9330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00016F745862898F,196201177000368,1,2009-04-12,2009-04-18,3900MB,26000.0,0.0,6.476809e+09,NaN,NaN,20090412,7866,0.0,1068.0,0.0,0.0,6.0,20090418,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00016F745862898F,196661177015632,1,2009-08-31,2009-09-02,3900HM,5000.0,0.0,6.119985e+08,6.119985e+08,NaN,20090831,6186,0.0,1068.0,0.0,0.0,2.0,20090902,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,6186,V5866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00016F745862898F,196091176981058,1,2009-09-17,2009-09-20,3913XU,5000.0,0.0,4.971603e+09,NaN,1.119000e+09,20090917,29590,0.0,1068.0,0.0,0.0,3.0,20090920,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00016F745862898F,196261176983265,1,2010-06-26,2010-07-01,3900MB,16000.0,0.0,6.408400e+09,1.960860e+09,NaN,20100626,5849,0.0,1100.0,0.0,0.0,5.0,20100701,983,3569,4019,3542,V8801,78820,2639,7840,7856,4271,NaN,NaN,E8889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
claims_df.DESYNPUF_ID.nunique()

37780

In [ ]:
claims_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66773 entries, 0 to 66772
Data columns (total 81 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   DESYNPUF_ID                     66773 non-null  object        
 1   CLM_ID                          66773 non-null  int64         
 2   SEGMENT                         66773 non-null  int64         
 3   CLM_FROM_DT                     66705 non-null  datetime64[ns]
 4   CLM_THRU_DT                     66705 non-null  datetime64[ns]
 5   PRVDR_NUM                       66773 non-null  object        
 6   CLM_PMT_AMT                     66773 non-null  float64       
 7   NCH_PRMRY_PYR_CLM_PD_AMT        66773 non-null  float64       
 8   AT_PHYSN_NPI                    66100 non-null  float64       
 9   OP_PHYSN_NPI                    39058 non-null  float64       
 10  OT_PHYSN_NPI                    7683 non-null   float64       
 11  CL

In [ ]:
claims_df.isna().sum()

DESYNPUF_ID                           0
CLM_ID                                0
SEGMENT                               0
CLM_FROM_DT                          68
CLM_THRU_DT                          68
PRVDR_NUM                             0
CLM_PMT_AMT                           0
NCH_PRMRY_PYR_CLM_PD_AMT              0
AT_PHYSN_NPI                        673
OP_PHYSN_NPI                      27715
OT_PHYSN_NPI                      59090
CLM_ADMSN_DT                          0
ADMTNG_ICD9_DGNS_CD                 599
CLM_PASS_THRU_PER_DIEM_AMT            0
NCH_BENE_IP_DDCTBL_AMT             2178
NCH_BENE_PTA_COINSRNC_LBLTY_AM        0
NCH_BENE_BLOOD_DDCTBL_LBLTY_AM        0
CLM_UTLZTN_DAY_CNT                   68
NCH_BENE_DSCHRG_DT                    0
CLM_DRG_CD                            0
ICD9_DGNS_CD_1                       95
ICD9_DGNS_CD_2                      526
ICD9_DGNS_CD_3                     1281
ICD9_DGNS_CD_4                     2768
ICD9_DGNS_CD_5                     5134


In [ ]:
!unzip 176541_DE1_0_2008_Beneficiary_Summary_File_Sample_1.zip
!unzip 176600_DE1_0_2009_Beneficiary_Summary_File_Sample_1.zip
!unzip 176537_DE1_0_2010_Beneficiary_Summary_File_Sample_20.zip

Archive:  176541_DE1_0_2008_Beneficiary_Summary_File_Sample_1.zip
replace DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  176600_DE1_0_2009_Beneficiary_Summary_File_Sample_1.zip
replace DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  176537_DE1_0_2010_Beneficiary_Summary_File_Sample_20.zip
replace DE1_0_2010_Beneficiary_Summary_File_Sample_20.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
summary_2008_df = pd.read_csv('DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv', parse_dates=['BENE_BIRTH_DT'])
summary_2009_df = pd.read_csv('DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv', parse_dates=['BENE_BIRTH_DT'])
summary_2010_df = pd.read_csv('DE1_0_2010_Beneficiary_Summary_File_Sample_20.csv', parse_dates=['BENE_BIRTH_DT'])

In [ ]:
summary_2008_df.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,PLAN_CVRG_MOS_NUM,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR
0,00013D2EFD8E45D1,1923-05-01,NaN,1,1,0,26,950,12,12,12,12,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,50.0,10.0,0.0,0.0,0.0,0.0
1,00016F745862898F,1943-01-01,NaN,1,1,0,39,230,12,12,0,0,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,700.0,240.0,0.0
2,0001FDD721E223DC,1936-09-01,NaN,2,1,0,39,280,12,12,0,12,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00021CA6FF03E670,1941-06-01,NaN,1,5,0,6,290,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00024B3D2352D2D0,1936-08-01,NaN,1,1,0,52,590,12,12,0,0,2,2,2,2,2,2,2,2,1,2,2,0.0,0.0,0.0,30.0,40.0,0.0,220.0,80.0,0.0


In [ ]:
summary_2008_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116352 entries, 0 to 116351
Data columns (total 32 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   DESYNPUF_ID               116352 non-null  object        
 1   BENE_BIRTH_DT             116352 non-null  datetime64[ns]
 2   BENE_DEATH_DT             1814 non-null    float64       
 3   BENE_SEX_IDENT_CD         116352 non-null  int64         
 4   BENE_RACE_CD              116352 non-null  int64         
 5   BENE_ESRD_IND             116352 non-null  object        
 6   SP_STATE_CODE             116352 non-null  int64         
 7   BENE_COUNTY_CD            116352 non-null  int64         
 8   BENE_HI_CVRAGE_TOT_MONS   116352 non-null  int64         
 9   BENE_SMI_CVRAGE_TOT_MONS  116352 non-null  int64         
 10  BENE_HMO_CVRAGE_TOT_MONS  116352 non-null  int64         
 11  PLAN_CVRG_MOS_NUM         116352 non-null  int64         
 12  SP

In [ ]:
summary_2008_df.isna().sum()

DESYNPUF_ID                      0
BENE_BIRTH_DT                    0
BENE_DEATH_DT               114538
BENE_SEX_IDENT_CD                0
BENE_RACE_CD                     0
BENE_ESRD_IND                    0
SP_STATE_CODE                    0
BENE_COUNTY_CD                   0
BENE_HI_CVRAGE_TOT_MONS          0
BENE_SMI_CVRAGE_TOT_MONS         0
BENE_HMO_CVRAGE_TOT_MONS         0
PLAN_CVRG_MOS_NUM                0
SP_ALZHDMTA                      0
SP_CHF                           0
SP_CHRNKIDN                      0
SP_CNCR                          0
SP_COPD                          0
SP_DEPRESSN                      0
SP_DIABETES                      0
SP_ISCHMCHT                      0
SP_OSTEOPRS                      0
SP_RA_OA                         0
SP_STRKETIA                      0
MEDREIMB_IP                      0
BENRES_IP                        0
PPPYMT_IP                        0
MEDREIMB_OP                      0
BENRES_OP                        0
PPPYMT_OP           

In [ ]:
summary_2008_df.shape

(116352, 32)

In [ ]:
summary_2008_df.DESYNPUF_ID.nunique()

116352

Combining the beneficiary data into a single dataframe.

In [ ]:
summary_df = pd.merge(summary_2009_df, summary_2009_df, how='outer')
summary_df = pd.merge(summary_df, summary_2010_df, how='outer')

In [ ]:
summary_df.shape

(227349, 32)

In [ ]:
summary_df.DESYNPUF_ID.nunique()

227349

In [ ]:
summary_df.DESYNPUF_ID.value_counts(ascending=False)

F88169F1D5A6C4DA    1
C637C041C4CDFF58    1
E5FE2734F033FB6F    1
8B0F61C80C6BABC0    1
E76903AB9F556050    1
                   ..
1C55D79082B9FC02    1
21EF380BCD773C96    1
9772AB46468E0C94    1
25A5B54C8A839FBC    1
FDABA0E9FAC30076    1
Name: DESYNPUF_ID, Length: 227349, dtype: int64

In [ ]:
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227349 entries, 0 to 227348
Data columns (total 32 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   DESYNPUF_ID               227349 non-null  object        
 1   BENE_BIRTH_DT             227349 non-null  datetime64[ns]
 2   BENE_DEATH_DT             3505 non-null    float64       
 3   BENE_SEX_IDENT_CD         227349 non-null  int64         
 4   BENE_RACE_CD              227349 non-null  int64         
 5   BENE_ESRD_IND             227349 non-null  object        
 6   SP_STATE_CODE             227349 non-null  int64         
 7   BENE_COUNTY_CD            227349 non-null  int64         
 8   BENE_HI_CVRAGE_TOT_MONS   227349 non-null  int64         
 9   BENE_SMI_CVRAGE_TOT_MONS  227349 non-null  int64         
 10  BENE_HMO_CVRAGE_TOT_MONS  227349 non-null  int64         
 11  PLAN_CVRG_MOS_NUM         227349 non-null  int64         
 12  SP

# Preprocess Data

In [ ]:
claims_df.shape

(66773, 81)

In [ ]:
claims_df['ADMTNG_ICD9_DGNS_CD'].isna().sum()

599

In [ ]:
# drop all lines with SEGMENT=2 because they contain no diagnosis codes nor procedure codes
claims_df = claims_df[claims_df['SEGMENT']==1]

In [ ]:
claims_df['ADMTNG_ICD9_DGNS_CD'].isna().sum()

531

In [ ]:
# set missing admitting diagnosis codes to first diagnosis code
missing_df = claims_df[claims_df['ADMTNG_ICD9_DGNS_CD'].isna()]
for idx, row in missing_df.iterrows():
  claims_df.at[idx, 'ADMTNG_ICD9_DGNS_CD'] = row.ICD9_DGNS_CD_1

In [ ]:
claims_df['ADMTNG_ICD9_DGNS_CD'].isna().sum()

1

In [ ]:
claims_df[claims_df['ADMTNG_ICD9_DGNS_CD'].isna()]

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,OT_PHYSN_NPI,CLM_ADMSN_DT,ADMTNG_ICD9_DGNS_CD,CLM_PASS_THRU_PER_DIEM_AMT,NCH_BENE_IP_DDCTBL_AMT,NCH_BENE_PTA_COINSRNC_LBLTY_AM,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,CLM_UTLZTN_DAY_CNT,NCH_BENE_DSCHRG_DT,CLM_DRG_CD,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,ICD9_DGNS_CD_3,ICD9_DGNS_CD_4,ICD9_DGNS_CD_5,ICD9_DGNS_CD_6,ICD9_DGNS_CD_7,ICD9_DGNS_CD_8,ICD9_DGNS_CD_9,ICD9_DGNS_CD_10,ICD9_PRCDR_CD_1,ICD9_PRCDR_CD_2,ICD9_PRCDR_CD_3,ICD9_PRCDR_CD_4,ICD9_PRCDR_CD_5,ICD9_PRCDR_CD_6,HCPCS_CD_1,HCPCS_CD_2,HCPCS_CD_3,HCPCS_CD_4,...,HCPCS_CD_6,HCPCS_CD_7,HCPCS_CD_8,HCPCS_CD_9,HCPCS_CD_10,HCPCS_CD_11,HCPCS_CD_12,HCPCS_CD_13,HCPCS_CD_14,HCPCS_CD_15,HCPCS_CD_16,HCPCS_CD_17,HCPCS_CD_18,HCPCS_CD_19,HCPCS_CD_20,HCPCS_CD_21,HCPCS_CD_22,HCPCS_CD_23,HCPCS_CD_24,HCPCS_CD_25,HCPCS_CD_26,HCPCS_CD_27,HCPCS_CD_28,HCPCS_CD_29,HCPCS_CD_30,HCPCS_CD_31,HCPCS_CD_32,HCPCS_CD_33,HCPCS_CD_34,HCPCS_CD_35,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45
26530,6448F1D0D8AB6C76,196481176963894,1,2009-01-30,2009-02-02,0700JR,4000.0,0.0,4.842646e+09,NaN,NaN,20090130,NaN,0.0,1068.0,0.0,0.0,3.0,20090202,058,NaN,8020,87343,V1582,42789,41401,49390,2859,V5861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
claims_df.at[26530, 'ADMTNG_ICD9_DGNS_CD'] = '8020' # set it to ICD9_DGNS_CD_2 value

In [ ]:
claims_df['ADMTNG_ICD9_DGNS_CD'].isna().sum()

0

In [ ]:
claims_df.CLM_ADMSN_DT.isna().sum()

0

In [ ]:
claims_df.NCH_BENE_DSCHRG_DT.isna().sum()

0

In [ ]:
claims_df.CLM_UTLZTN_DAY_CNT.isna().sum()

0

In [ ]:
claims_df.CLM_UTLZTN_DAY_CNT.value_counts()

3.0      10896
2.0       9542
1.0       8130
4.0       7936
5.0       5975
6.0       4449
7.0       3482
8.0       2607
0.0       2266
9.0       1892
10.0      1472
11.0      1236
12.0       972
13.0       822
14.0       738
15.0       598
16.0       477
17.0       382
18.0       294
21.0       274
19.0       269
20.0       249
22.0       192
23.0       154
24.0       129
25.0       120
26.0       108
28.0       105
27.0        97
29.0        77
30.0        74
33.0        65
31.0        62
32.0        60
35.0        48
37.0        39
34.0        35
36.0        35
39.0        25
41.0        23
42.0        22
38.0        21
44.0        16
45.0        16
40.0        16
43.0        15
50.0        14
48.0        12
49.0        11
47.0         9
46.0         8
54.0         7
56.0         7
53.0         7
51.0         7
71.0         6
64.0         6
62.0         6
79.0         5
60.0         5
57.0         5
58.0         5
69.0         4
55.0         4
67.0         4
63.0         4
52.0      

In [ ]:
claims_sub_df = claims_df[['DESYNPUF_ID', 'ADMTNG_ICD9_DGNS_CD','CLM_UTLZTN_DAY_CNT']]
claims_sub_df.isna().sum()

DESYNPUF_ID            0
ADMTNG_ICD9_DGNS_CD    0
CLM_UTLZTN_DAY_CNT     0
dtype: int64

In [ ]:
summary_sub_df = summary_df[['DESYNPUF_ID', 'BENE_BIRTH_DT', 'BENE_SEX_IDENT_CD', 'SP_ALZHDMTA', 'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_COPD',
                             'SP_DEPRESSN', 'SP_DIABETES', 'SP_ISCHMCHT', 'SP_OSTEOPRS', 'SP_RA_OA', 'SP_STRKETIA']]
summary_sub_df.isna().sum()

DESYNPUF_ID          0
BENE_BIRTH_DT        0
BENE_SEX_IDENT_CD    0
SP_ALZHDMTA          0
SP_CHF               0
SP_CHRNKIDN          0
SP_CNCR              0
SP_COPD              0
SP_DEPRESSN          0
SP_DIABETES          0
SP_ISCHMCHT          0
SP_OSTEOPRS          0
SP_RA_OA             0
SP_STRKETIA          0
dtype: int64

In [ ]:
data_df = claims_sub_df.merge(summary_sub_df, on='DESYNPUF_ID')
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66518 entries, 0 to 66517
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DESYNPUF_ID          66518 non-null  object        
 1   ADMTNG_ICD9_DGNS_CD  66518 non-null  object        
 2   CLM_UTLZTN_DAY_CNT   66518 non-null  float64       
 3   BENE_BIRTH_DT        66518 non-null  datetime64[ns]
 4   BENE_SEX_IDENT_CD    66518 non-null  int64         
 5   SP_ALZHDMTA          66518 non-null  int64         
 6   SP_CHF               66518 non-null  int64         
 7   SP_CHRNKIDN          66518 non-null  int64         
 8   SP_CNCR              66518 non-null  int64         
 9   SP_COPD              66518 non-null  int64         
 10  SP_DEPRESSN          66518 non-null  int64         
 11  SP_DIABETES          66518 non-null  int64         
 12  SP_ISCHMCHT          66518 non-null  int64         
 13  SP_OSTEOPRS          66518 non-

In [ ]:
data_df['year'] = pd.DatetimeIndex(data_df['BENE_BIRTH_DT']).year
data_df['age'] = 2020-data_df['year']
data_df.drop(['year','BENE_BIRTH_DT'], axis='columns', inplace=True)

CLM_UTLZTN_DAY_CNT will be the target to predict and ADMTNG_ICD9_DGNS_CD will be the high cardinality categorical column we encode. Renaming them for convenience.

In [ ]:
data_df = data_df.rename(columns={"CLM_UTLZTN_DAY_CNT": "target", "ADMTNG_ICD9_DGNS_CD": "diagnosis"})
data_df.head()

,DESYNPUF_ID,diagnosis,target,BENE_SEX_IDENT_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,age
0,00013D2EFD8E45D1,4580,1.0,1,2,2,2,2,2,2,2,2,2,2,2,97
1,00016F745862898F,7866,6.0,1,1,2,1,2,2,1,1,1,2,1,1,77
2,00016F745862898F,6186,2.0,1,1,2,1,2,2,1,1,1,2,1,1,77
3,00016F745862898F,29590,3.0,1,1,2,1,2,2,1,1,1,2,1,1,77
4,00016F745862898F,5849,5.0,1,1,2,1,2,2,1,1,1,2,1,1,77


In [ ]:
# Note the high cardinality
data_df.diagnosis.nunique()

2316

We will target encode these 2316 unique values in the diagnosis column.

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=seed)

In [ ]:
y_train = train_df.target.values
y_test = test_df.target.values

In [ ]:
!pip install category_encoders -q

In [ ]:
from category_encoders import TargetEncoder
encoder = TargetEncoder(cols=['diagnosis'])
train_df = encoder.fit_transform(train_df, y_train)
test_df = encoder.transform(test_df)

In [ ]:
train_df.sample(10)

,DESYNPUF_ID,diagnosis,target,BENE_SEX_IDENT_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,age
5627,15028A2D722313C3,14.307639,13.0,1,1,1,1,2,2,1,1,1,2,2,2,37
64326,F7D9A4922DAAE1D8,4.216216,4.0,1,1,1,1,2,1,1,1,1,2,2,2,72
20130,4BE7EC634A5E6514,12.375000,12.0,2,1,1,1,2,1,1,1,1,2,1,2,101
19126,4857186379E19600,7.194030,8.0,1,2,1,2,1,2,1,1,1,2,1,2,87
11795,2C94EB5DFB1E53BE,7.194030,17.0,2,1,1,1,2,1,1,1,1,2,2,2,61
25999,62CDC12D5520410B,5.546122,3.0,1,2,1,1,2,2,1,1,1,2,2,2,83
2184,07D80D6696BFF0C2,4.948905,4.0,2,2,1,2,2,2,2,1,1,1,1,2,94
34022,81AC20CBEBE214B3,3.667130,2.0,2,1,2,2,2,2,2,1,1,2,2,2,82
34419,830294D70A8112C0,5.891821,3.0,2,2,1,1,2,2,2,1,1,1,2,2,82
56625,D9D28F7613F996ED,3.667130,5.0,2,2,2,2,2,2,2,2,1,2,2,2,93


In [ ]:
x_train = train_df.drop(['target', 'DESYNPUF_ID'], axis='columns').values
x_test = test_df.drop(['target', 'DESYNPUF_ID'], axis='columns').values

# The Model

In [ ]:
tf.keras.backend.clear_session()
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input, Model

inp = Input(shape=(x_train.shape[1]))
x = Dense(512, activation='relu')(inp)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
out = Dense(1)(x)

model = Model(inputs=[inp], outputs=[out])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "functional_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 14)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               7680      
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_13 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 65        
Total params: 180,225
Trainable params: 180,225
Non-trainable params: 0
________________________________________________

# Training

Since the data is synthetic we should not expect any kind of meaning in our results. But just running 10 epochs to demonstrate how to accomplish this with real data.

In [ ]:
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1663/1663 [==============================] - 7s 4ms/step - loss: 36.2417 - mae: 3.5798 - val_loss: 35.0276 - val_mae: 3.5725
Epoch 2/10
1663/1663 [==============================] - 7s 4ms/step - loss: 35.1941 - mae: 3.5139 - val_loss: 35.8047 - val_mae: 3.3381
Epoch 3/10
1663/1663 [==============================] - 7s 4ms/step - loss: 35.2045 - mae: 3.5098 - val_loss: 35.0267 - val_mae: 3.5578
Epoch 4/10
1663/1663 [==============================] - 7s 4ms/step - loss: 35.0808 - mae: 3.5012 - val_loss: 34.9816 - val_mae: 3.5025
Epoch 5/10
1663/1663 [==============================] - 7s 4ms/step - loss: 34.9721 - mae: 3.4949 - val_loss: 35.3738 - val_mae: 3.7100
Epoch 6/10
1663/1663 [==============================] - 7s 4ms/step - loss: 34.9603 - mae: 3.4952 - val_loss: 34.9381 - val_mae: 3.5747
Epoch 7/10
1663/1663 [==============================] - 7s 4ms/step - loss: 34.9490 - mae: 3.4960 - val_loss: 34.9946 - val_mae: 3.4306
Epoch 8/10
1663/1663 [==========================